# 이진 벡터
- 기존의 고차원 부동소수점 벡터를 0과 1만 포함하는 이진 벡터로 변환하는 특별한 형태의 데이터 표현
- 벡터의 크기를 압축할 뿐만 아니라 의미 정보를 유지하면서 저장 및 계산 비용을 줄여줌
- 중요하지 않은 특징에 대한 정밀도가 필수적이지 않은 경우, 이진 벡터는 원래 부동소수점 벡터의 무결성과 유용성 대부분을 효과적으로 유지할 수 있음
- 검색 엔진이나 추천 시스템과 같은 대규모 AI 시스템에서는 방대한 양의 데이터를 실시간으로 처리하는 것이 핵심
- 메모리와 처리 능력이 제한된 모바일 디바이스나 임베디드 시스템과 같은 리소스 제약이 있는 환경에서 유용

참고 자료: https://milvus.io/docs/ko/binary-vector.md

## 개요
- 이진 벡터는 이미지, 텍스트 또는 오디오와 같은 복잡한 객체를 고정 길이의 이진 값으로 인코딩하는 방법
- 8차원 바이너리는 [1, 0, 1, 1, 0, 0, 1, 0]으료 표현

### 이진 벡터 특징
- 효율적인 저장 공간: 각 차원에는 1비트만 저장 공간이 필요해 저장 공간이 크게 줄어듦
- 빠른 계산: XOR과 같은 비트 연산을 사용해 벡터 간의 유사도를 빠르게 계산
- 고정 길이: 원본 텍스트 길이에 관계없이 벡터의 길이가 일정하게 유지되므로 색인 및 검색이 더 쉬워짐
- 간단하고 직관적: 키워드의 존재 여부를 직접 반영하므로 특정 전문 검색 작업에 적합
- 이미지 처리의 경우 지각 해싱 알고리즘을 통해 이미지의 이진 특징을 생성할 수 있음

# 벡터 필드 추가
- datatype을 지원되는 바이너리 벡터 데이터 유형(예: BINARY_VECTOR)로 설정
- dim 매개변수를 사용하여 벡터의 차원을 지정, 삽입할 때 바이너리 벡터를 바이트 배열로 변환해야 하므로 dim은 8의 배수여야 함
- 8개의 부울 값(0 또는 1)은 모두 1바이트로 패킹됨, example) dim=128일경우 16바이트 배열이 필요

In [1]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(uri="http://localhost:19530")

schema = client.create_schema(
    auto_id=True,
    enable_dynamic_fields=True,
)

schema.add_field(field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100)
schema.add_field(field_name="binary_vector", datatype=DataType.BINARY_VECTOR, dim=128)


{'auto_id': True, 'description': '', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'binary_vector', 'description': '', 'type': <DataType.BINARY_VECTOR: 100>, 'params': {'dim': 128}}], 'enable_dynamic_field': False}

## 벡터 필드에 대한 인덱스 매개변수 설정
- 인덱스 설명: https://milvus.io/docs/ko/index-explained.md
- 메트릭 유형: https://milvus.io/docs/ko/metric.md

In [ ]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="binary_vector",
    index_name="binary_vector_index",
    index_type="AUTOINDEX",
    metric_type="HAMMING"
)

# 컬렉션 만들기

In [4]:
client.create_collection(
    collection_name="my_collection2",
    schema=schema,
    index_params=index_params
)

# 데이터 삽입

In [6]:
def convert_bool_list_to_bytes(bool_list):
    if len(bool_list) % 8 != 0:
        raise ValueError("The length of a boolean list must be a multiple of 8")
    
    byte_array = bytearray(len(bool_list) // 8)
    for i, bit in enumerate(bool_list):
        if bit == 1:
            index = i // 8
            shift = i % 8
            byte_array[index] |= (1 << shift)
    return bytes(byte_array)

bool_vectors = [
    [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0] + [0] * 112,
    [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1] + [0] * 112,
]

data = [{"binary_vector": convert_bool_list_to_bytes(bool_vector) for bool_vector in bool_vectors}]

client.insert(
    collection_name="my_collection2",
    data=data
)

{'insert_count': 1, 'ids': ['459127794345938950'], 'cost': 0}

# 유사도 검색 수행

In [8]:
search_params = {
    "params": {"nprobe": 10},  # nprobe는 검색 시 고려할 인덱스의 개수
}

query_bool_list = [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0] + [0] * 112
query_vector = convert_bool_list_to_bytes(query_bool_list)

res = client.search(
    collection_name="my_collection2",
    data=[query_vector],
    anns_field="binary_vector",
    search_params=search_params,
    limit=5,
    output_fields=["pk"]
)

In [9]:
print(res)

data: [[{'pk': '459127794345938950', 'distance': 10.0, 'entity': {'pk': '459127794345938950'}}]]
